In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk, ImageDraw, ImageFont
import os
import textwrap

class MemeGeneratorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Generador de Memes")
        self.root.geometry("800x600")
        
        # Variables para almacenar las imágenes y textos
        self.image_paths = []
        self.current_image_index = 0
        self.image = None
        self.tk_image = None
        self.font_path = "arial.ttf"  # Fuente por defecto (puedes cambiarla)
        
        # Configurar la interfaz
        self.setup_ui()
        
    def setup_ui(self):
        # Frame principal
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Panel de visualización de imagen
        self.image_panel = tk.Label(main_frame)
        self.image_panel.pack(fill=tk.BOTH, expand=True)
        
        # Frame de controles
        control_frame = tk.Frame(main_frame)
        control_frame.pack(fill=tk.X, pady=10)
        
        # Botones para cargar imágenes
        btn_load = tk.Button(control_frame, text="Cargar Imágenes", command=self.load_images)
        btn_load.pack(side=tk.LEFT, padx=5)
        
        btn_prev = tk.Button(control_frame, text="< Anterior", command=self.prev_image)
        btn_prev.pack(side=tk.LEFT, padx=5)
        
        btn_next = tk.Button(control_frame, text="Siguiente >", command=self.next_image)
        btn_next.pack(side=tk.LEFT, padx=5)
        
        # Controles de texto
        text_frame = tk.Frame(main_frame)
        text_frame.pack(fill=tk.X, pady=5)
        
        tk.Label(text_frame, text="Texto Superior:").pack(side=tk.LEFT)
        self.top_text = tk.Entry(text_frame)
        self.top_text.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
        
        tk.Label(text_frame, text="Texto Inferior:").pack(side=tk.LEFT)
        self.bottom_text = tk.Entry(text_frame)
        self.bottom_text.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
        
        # Controles de estilo
        style_frame = tk.Frame(main_frame)
        style_frame.pack(fill=tk.X, pady=5)
        
        tk.Label(style_frame, text="Tamaño Fuente:").pack(side=tk.LEFT)
        self.font_size = tk.Scale(style_frame, from_=10, to=100, orient=tk.HORIZONTAL)
        self.font_size.set(40)
        self.font_size.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
        
        tk.Label(style_frame, text="Color:").pack(side=tk.LEFT)
        self.text_color = tk.Entry(style_frame)
        self.text_color.insert(0, "white")
        self.text_color.pack(side=tk.LEFT, padx=5)
        
        # Botón para generar meme
        btn_generate = tk.Button(main_frame, text="Generar Meme", command=self.generate_meme)
        btn_generate.pack(pady=10)
        
        # Botón para guardar meme
        btn_save = tk.Button(main_frame, text="Guardar Meme", command=self.save_meme)
        btn_save.pack(pady=5)
        
        # Mensaje inicial
        self.show_message("Carga imágenes y edita los textos para crear tus memes!")
    
    def load_images(self):
        file_paths = filedialog.askopenfilenames(
            title="Seleccionar imágenes",
            filetypes=[("Imágenes", "*.jpg *.jpeg *.png *.gif *.bmp")]
        )
        
        if file_paths:
            self.image_paths = list(file_paths)
            self.current_image_index = 0
            self.display_current_image()
    
    def display_current_image(self):
        if 0 <= self.current_image_index < len(self.image_paths):
            try:
                self.image = Image.open(self.image_paths[self.current_image_index])
                self.update_image_display()
            except Exception as e:
                messagebox.showerror("Error", f"No se pudo cargar la imagen: {str(e)}")
    
    def update_image_display(self):
        if self.image:
            # Redimensionar para mostrar en la GUI manteniendo relación de aspecto
            display_width = 600
            display_height = 400
            img_ratio = self.image.width / self.image.height
            if img_ratio > display_width / display_height:
                new_width = display_width
                new_height = int(display_width / img_ratio)
            else:
                new_height = display_height
                new_width = int(display_height * img_ratio)
            
            resized_img = self.image.resize((new_width, new_height), Image.LANCZOS)
            self.tk_image = ImageTk.PhotoImage(resized_img)
            self.image_panel.config(image=self.tk_image)
    
    def prev_image(self):
        if len(self.image_paths) > 0:
            self.current_image_index = (self.current_image_index - 1) % len(self.image_paths)
            self.display_current_image()
    
    def next_image(self):
        if len(self.image_paths) > 0:
            self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)
            self.display_current_image()
    
    def generate_meme(self):
        if not self.image:
            messagebox.showwarning("Advertencia", "Por favor carga una imagen primero.")
            return
        
        try:
            # Crear una copia de la imagen para editar
            meme_img = self.image.copy()
            draw = ImageDraw.Draw(meme_img)
            
            # Configurar la fuente
            font_size = self.font_size.get()
            try:
                font = ImageFont.truetype(self.font_path, font_size)
            except:
                font = ImageFont.load_default()
            
            # Obtener textos
            top_text = self.top_text.get()
            bottom_text = self.bottom_text.get()
            
            # Obtener color (blanco por defecto)
            color = self.text_color.get().strip().lower()
            
            # Añadir texto superior
            if top_text:
                self.add_text_to_image(draw, meme_img, font, top_text, "top", color)
            
            # Añadir texto inferior
            if bottom_text:
                self.add_text_to_image(draw, meme_img, font, bottom_text, "bottom", color)
            
            # Actualizar la visualización
            self.image = meme_img
            self.update_image_display()
            
        except Exception as e:
            messagebox.showerror("Error", f"Error al generar el meme: {str(e)}")
    
    def add_text_to_image(self, draw, image, font, text, position, color="white"):
        img_width, img_height = image.size
        margin = 10
        
        # Ajustar texto para que quepa en la imagen
        avg_char_width = sum(font.getlength(char) for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789") / 62
        max_chars = int(img_width * 0.9 / avg_char_width)
        text_lines = textwrap.wrap(text, width=max_chars)
        
        # Calcular posición vertical
        if position == "top":
            y = margin
        else:  # bottom
            total_text_height = len(text_lines) * (font.size + 5)
            y = img_height - total_text_height - margin
        
        # Dibujar cada línea de texto
        for line in text_lines:
            line_width = font.getlength(line)
            x = (img_width - line_width) / 2
            
            # Añadir borde negro para mejor legibilidad
            for x_offset in [-2, 0, 2]:
                for y_offset in [-2, 0, 2]:
                    draw.text((x + x_offset, y + y_offset), line, font=font, fill="black")
            
            # Texto principal
            draw.text((x, y), line, font=font, fill=color)
            
            y += font.size + 5
    
    def save_meme(self):
        if not self.image:
            messagebox.showwarning("Advertencia", "No hay meme para guardar.")
            return
        
        file_path = filedialog.asksaveasfilename(
            title="Guardar meme",
            defaultextension=".png",
            filetypes=[("PNG", "*.png"), ("JPEG", "*.jpg"), ("Todos los archivos", "*.*")]
        )
        
        if file_path:
            try:
                self.image.save(file_path)
                messagebox.showinfo("Éxito", f"Meme guardado correctamente en:\n{file_path}")
            except Exception as e:
                messagebox.showerror("Error", f"No se pudo guardar el meme: {str(e)}")
    
    def show_message(self, message):
        self.image_panel.config(text=message, font=("Arial", 14), fg="gray")

if __name__ == "__main__":
    root = tk.Tk()
    app = MemeGeneratorApp(root)
    root.mainloop()